In [8]:
# Import necessary libraries
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to SQLite database (or create it)
conn = sqlite3.connect('school_platform.db')
cursor = conn.cursor()


In [9]:



# Load data from Excel file
excel_file = 'Latest TechTest-BI-Dataset 2021.xlsx'
teacher_activity = pd.read_excel(excel_file, sheet_name='Teacher Activity')
student_activity = pd.read_excel(excel_file, sheet_name='Student Activity')
data_dictionary = pd.read_excel(excel_file, sheet_name='Data Dictonary')



In [10]:

# Write data to SQLite tables
teacher_activity.to_sql('Teacher_Activity', conn, if_exists='replace', index=False)
student_activity.to_sql('Student_Activity', conn, if_exists='replace', index=False)




72516

In [12]:

# Top 5 schools with overall teachers’ login% > 60%
query1 = """
SELECT 
    school_name,
    COUNT(DISTINCT teacher_id) AS total_teachers,
    COUNT(DISTINCT CASE WHEN login_date IS NOT NULL THEN teacher_id END) AS teachers_logged_in,
    (CAST(COUNT(DISTINCT CASE WHEN login_date IS NOT NULL THEN teacher_id END) AS FLOAT) / COUNT(DISTINCT teacher_id)) * 100 AS login_percentage
FROM 
    Teacher_Activity
GROUP BY 
    school_id
HAVING 
    (CAST(COUNT(DISTINCT CASE WHEN login_date IS NOT NULL THEN teacher_id END) AS FLOAT) / COUNT(DISTINCT teacher_id)) * 100 > 60
ORDER BY 
    login_percentage DESC
LIMIT 5;
"""
top_schools = pd.read_sql_query(query1, conn)
print("Top 5 Schools with Overall Teachers’ Login% > 60%:")
print(top_schools)


DatabaseError: Execution failed on sql '
SELECT 
    school_name,
    COUNT(DISTINCT teacher_id) AS total_teachers,
    COUNT(DISTINCT CASE WHEN login_date IS NOT NULL THEN teacher_id END) AS teachers_logged_in,
    (CAST(COUNT(DISTINCT CASE WHEN login_date IS NOT NULL THEN teacher_id END) AS FLOAT) / COUNT(DISTINCT teacher_id)) * 100 AS login_percentage
FROM 
    Teacher_Activity
GROUP BY 
    school_id
HAVING 
    (CAST(COUNT(DISTINCT CASE WHEN login_date IS NOT NULL THEN teacher_id END) AS FLOAT) / COUNT(DISTINCT teacher_id)) * 100 > 60
ORDER BY 
    login_percentage DESC
LIMIT 5;
': no such column: login_date

In [ ]:

# Teachers Login day over day change in percentage across all schools
query2 = """
WITH daily_login AS (
    SELECT 
        school_id,
        login_date,
        COUNT(DISTINCT teacher_id) AS daily_logins
    FROM 
        Teacher_Activity
    GROUP BY 
        school_id, login_date
),
daily_change AS (
    SELECT 
        school_id,
        login_date,
        daily_logins,
        LAG(daily_logins) OVER (PARTITION BY school_id ORDER BY login_date) AS prev_day_logins,
        (daily_logins - LAG(daily_logins) OVER (PARTITION BY school_id ORDER BY login_date)) * 100.0 / LAG(daily_logins) OVER (PARTITION BY school_id ORDER BY login_date) AS day_over_day_change
    FROM 
        daily_login
)
SELECT 
    school_id,
    login_date,
    day_over_day_change
FROM 
    daily_change;
"""
day_over_day_change = pd.read_sql_query(query2, conn)
print("Teachers Login Day Over Day Change in Percentage Across All Schools:")
print(day_over_day_change)


In [ ]:

# If each billable student pays $500/Month, what’s the revenue generated per school
query3 = """
WITH monthly_login AS (
    SELECT 
        school_id,
        student_id,
        COUNT(DISTINCT login_date) AS login_days
    FROM 
        Student_Activity
    GROUP BY 
        school_id, student_id
),
billable_students AS (
    SELECT 
        school_id,
        COUNT(DISTINCT student_id) AS billable_students
    FROM 
        monthly_login
    WHERE 
        login_days >= 1
    GROUP BY 
        school_id
)
SELECT 
    school_id,
    billable_students * 500 AS revenue_per_school
FROM 
    billable_students;
"""
revenue_per_school = pd.read_sql_query(query3, conn)
print("Revenue Generated per School:")
print(revenue_per_school)


In [ ]:

# Find the number of teachers per school who logged in 3 consecutive days
query4 = """
WITH teacher_login_dates AS (
    SELECT 
        teacher_id,
        school_id,
        login_date,
        LEAD(login_date, 1) OVER (PARTITION BY teacher_id ORDER BY login_date) AS next_login,
        LEAD(login_date, 2) OVER (PARTITION BY teacher_id ORDER BY login_date) AS next_next_login
    FROM 
        Teacher_Activity
),
consecutive_logins AS (
    SELECT 
        teacher_id,
        school_id
    FROM 
        teacher_login_dates
    WHERE 
        JULIANDAY(next_login) - JULIANDAY(login_date) = 1 
        AND JULIANDAY(next_next_login) - JULIANDAY(next_login) = 1
    GROUP BY 
        teacher_id, school_id
)
SELECT 
    school_id,
    COUNT(DISTINCT teacher_id) AS teachers_with_3_consecutive_logins
FROM 
    consecutive_logins
GROUP BY 
    school_id;
"""
teachers_consecutive_logins = pd.read_sql_query(query4, conn)
print("Number of Teachers per School Who Logged in 3 Consecutive Days:")
print(teachers_consecutive_logins)


In [ ]:

# Find the weekly average for student login activity per school
query5 = """
WITH weekly_logins AS (
    SELECT 
        school_id,
        strftime('%Y-%W', login_date) AS week,
        COUNT(DISTINCT student_id) AS weekly_student_logins
    FROM 
        Student_Activity
    GROUP BY 
        school_id, week
)
SELECT 
    school_id,
    AVG(weekly_student_logins) AS average_weekly_logins
FROM 
    weekly_logins
GROUP BY 
    school_id;
"""
weekly_average_logins = pd.read_sql_query(query5, conn)
print("Weekly Average for Student Login Activity per School:")
print(weekly_average_logins)


In [ ]:

# Data visualization

# Top 5 Schools with Overall Teachers’ Login% > 60%
plt.figure(figsize=(10, 6))
sns.barplot(x='school_id', y='login_percentage', data=top_schools)
plt.title('Top 5 Schools with Overall Teachers’ Login% > 60%')
plt.xlabel('School ID')
plt.ylabel('Login Percentage')
plt.show()

# Teachers Login Day Over Day Change in Percentage Across All Schools
plt.figure(figsize=(12, 6))
for school in day_over_day_change['school_id'].unique():
    data = day_over_day_change[day_over_day_change['school_id'] == school]
    plt.plot(data['login_date'], data['day_over_day_change'], label=f'School {school}')
plt.title('Teachers Login Day Over Day Change in Percentage Across All Schools')
plt.xlabel('Date')
plt.ylabel('Day Over Day Change (%)')
plt.legend()
plt.show()

# Revenue Generated per School
plt.figure(figsize=(10, 6))
sns.barplot(x='school_id', y='revenue_per_school', data=revenue_per_school)
plt.title('Revenue Generated per School')
plt.xlabel('School ID')
plt.ylabel('Revenue ($)')
plt.show()

# Number of Teachers per School Who Logged in 3 Consecutive Days
plt.figure(figsize=(10, 6))
sns.barplot(x='school_id', y='teachers_with_3_consecutive_logins', data=teachers_consecutive_logins)
plt.title('Number of Teachers per School Who Logged in 3 Consecutive Days')
plt.xlabel('School ID')
plt.ylabel('Number of Teachers')
plt.show()

# Weekly Average for Student Login Activity per School
plt.figure(figsize=(10, 6))
sns.barplot(x='school_id', y='average_weekly_logins', data=weekly_average_logins)
plt.title('Weekly Average for Student Login Activity per School')
plt.xlabel('School ID')
plt.ylabel('Average Weekly Logins')
plt.show()

# Close the database connection
conn.close()
